In [1]:
import rasterio as rio
import pandas as pd, json
import numpy as np
import time
import geojson

### Georeferencing

In [ ]:
gdal_translate -of GTiff -gcp 11.4625 6.2766 -180 90 -gcp 11.4625 4800.7 -180 -90 -gcp 4819.45 4814.26 180 -90 -gcp 4805.89 6.2766 180 90 "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/IRRIG_ASC_files/result_threshold.tif" "/private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result_threshold.tif"
gdalwarp -r near -tps -co COMPRESS=NONE  -t_srs EPSG:4326 "/private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result_threshold.tif" "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/IRRIG_ASC_files/result_threshold_modified.tif"


In [140]:
# # v2
# !gdal_translate -of GTiff -gcp 0 6.2766 -180 90 -gcp 0 4800.7 -180 -90 -gcp 4805.89 4814.26 180 -90 -gcp 4805.89 6.2766 180 90 "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/IRRIG_ASC_files/result_threshold.tif" "/private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result_threshold.tif"
# v3
!gdal_translate -of GTiff -gcp 0 0 -180 90 -gcp 0 4800 -180 -90 -gcp 4800 4800 180 -90 -gcp 4800 0 180 90 "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/RESULTS/2019/result_2019_12.tif" "/private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result.tif"
# !gdal_translate -of GTiff -gcp 0 0 -180 90 -gcp 0 4800 -180 -90 -gcp 4800 4800 180 -90 -gcp 4800 0 180 90 "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/IRRIG_ASC_files/result_50.tif" "/private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result.tif"

Input file size is 4800, 4800
0...10...20...30...40...50...60...70...80...90...100 - done.


In [141]:
!gdalwarp -r near -tps -co COMPRESS=NONE  -t_srs EPSG:4326 "/private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result.tif" "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/RESULTS/2019/result_2019_012.tif"

Creating output file that is 6072P x 3036L.
Processing /private/var/folders/9k/25jfq71d70v_hh04cjjb94jr0000gp/T/result.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


### tif to asc

In [142]:
!gdal_translate -of GTiff "/Users/kaiqi/Desktop/W210/monthly_growing_area_grids/RESULTS/2019/result_2019_012.tif" "result_2019_012.asc"

Input file size is 6072, 3036
0...10...20...30...40...50...60...70...80...90...100 - done.


In [143]:
src = rio.open("result_2019_001.asc")
data = src.read()
mosaic = data[0]

In [144]:
print(mosaic.shape)

(3036, 6072)


In [145]:
mosaic[mosaic>0]

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

### Get xyz of georeferenced asc file

In [14]:
!gdal_translate -of xyz -co ADD_HEADER_LINE=YES -co COLUMN_SEPARATOR="," "result_01_001.asc" xyz_result_2008.csv

Input file size is 6072, 3036
0...10...20...30...40...50...60...70...80...90...100 - done.


### Steps
1. Turn the lat-lon into dataframe 
2. Get the data array from the asc file, flatten array
3. Combine lat-lon and data array, Turn into df
4. Add in month column to array 
5. Filter for rows where irrig = 1
6. Use df to geojson function


In [60]:
### 1. Get the lat-lon from xyz file 
# xyz_coord = pd.read_csv("xyz_points.csv") # labelled dataset 2160 x 4320
xyz_coord = pd.read_csv("xyz_result_2008.csv") # xyz for result_threshold_modified.asc
xyz_coord = xyz_coord.rename(columns={'X':'longitude', 'Y':'latitude'})

In [61]:
xyz_coord.shape
xyz_coord.head()

,longitude,latitude,Z
0,-179.970354,89.970354,0
1,-179.911061,89.970354,0
2,-179.851768,89.970354,0
3,-179.792476,89.970354,0
4,-179.733183,89.970354,0


In [99]:
xyz_coord.longitude = xyz_coord.longitude.round(decimals=2)
xyz_coord.latitude = xyz_coord.latitude.round(decimals=2)
xyz_coord.head()

,longitude,latitude,Z
0,-179.97,89.97,0
1,-179.91,89.97,0
2,-179.85,89.97,0
3,-179.79,89.97,0
4,-179.73,89.97,0


In [63]:
month_dict = {'001': 0,'002': 1,'003': 2,'004': 3,'005': 4,'006': 5,
              '007': 6,'008': 7,'009': 8,'010': 9,'011': 10,'012': 11}

def process(filelist):
    
    final_df = pd.DataFrame()
    for file in filelist:
        
        month_no = file[-7:-4:1]
        
        src = rio.open(file)
        data = src.read()
        mosaic = data[0]
#         mosaic_replace = np.where(mosaic > 0, 1, 0).flatten() # for result tifs
#         mosaic_replace = np.where(mosaic > 10, 1, 0).flatten() # use this for labeled dataset
        mosaic_replace = mosaic.flatten()
    
        mosaic_df = pd.DataFrame(data=mosaic_replace)
        mosaic_df = mosaic_df.rename(columns={0: "irrig"})
        comb_df = pd.concat([xyz_coord, mosaic_df], axis=1)
        
        comb_filtered = comb_df[comb_df.irrig != 0]
        comb_filtered['month'] = [month_dict[month_no] for x in list(range(len(comb_filtered)))]
        comb_alt = comb_filtered.iloc[::5, :]  # get every nth point
#         comb_alt = comb_filtered.iloc[::10, :]  # get every nth point
#         comb_alt = comb_filtered
    
        final_df = pd.concat([final_df, comb_alt], ignore_index=True)
    
    return final_df

In [97]:
file = 'result_25_001.asc'
src = rio.open(file)
data = src.read()
mosaic = data[0]
mosaic[mosaic>0]
# mosaic_replace = np.where(mosaic > 0, 1, 0).flatten() 
# mosaic_replace[mosaic_replace>0]

array([65535, 65535, 65535, ..., 65535, 65535, 65535], dtype=uint16)

In [64]:
# code from https://geoffboeing.com/2015/10/exporting-python-data-geojson/
# check out https://github.com/gboeing/urban-data-science/blob/master/17-Leaflet-Web-Mapping/leaflet-simple-demo/pandas-to-geojson.ipynb
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [147]:
# files = ['result_2019_001.asc']
# files = ['result_25_001.asc']
files = ["result_2019_001.asc","result_2019_002.asc","result_2019_003.asc"]
# files =["allcrop_irrigated_011.asc","allcrop_irrigated_012.asc"]

output_filename = 'pred_2019_001_002_003.geojson'

result_df = process(files)
print(result_df.head(-10))
print("Start converting to geojson...")

cols = ['irrig','month']
start = time.time()
geojson_dict = df_to_geojson(result_df, properties=cols)

end = time.time()
print("Complete. Time taken in s", end - start)

geojson_str = json.dumps(geojson_dict, indent=2)

with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))

/Users/kaiqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


        longitude  latitude  Z  irrig  month
0            9.77     58.84  1    1.0      0
1           10.06     58.84  1    1.0      0
2           10.36     58.84  1    1.0      0
3            9.88     58.78  1    1.0      0
4           10.18     58.78  1    1.0      0
...           ...       ... ..    ...    ...
385539     -69.63    -49.72  0    1.0      2
385540     -69.33    -49.72  0    1.0      2
385541     -70.16    -49.78  0    1.0      2
385542     -69.86    -49.78  0    1.0      2
385543     -69.57    -49.78  0    1.0      2

[385544 rows x 5 columns]
Start converting to geojson...
Complete. Time taken in s 31.394718885421753
